In [67]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [68]:
import numpy as np
import pandas as pd
import time 
import matplotlib.pyplot as plt
import seaborn as sns

In [69]:
import os
FEATURES_PATH = os.path.join("/Data", "myData.csv")
print (FEATURES_PATH)

import pandas as pd


def load_tracks_features_data(tracks_features=FEATURES_PATH):
    #csv_path = os.path.join("Data","myData.csv")
    csv_path='/Users/ruddirodriguez/Dropbox/Machine_Learning/Tracks_Classification/Data/myData1.csv'
    return pd.read_csv(csv_path)
features_tracks_origi = load_tracks_features_data()

/Data/myData.csv


In [70]:
features_tracks = features_tracks_origi[list(features_tracks_origi.columns[0:10])] 
features_tracks.head()

,Id,netDisp,efficiency,straightness,Asym,bending,pointSkew,pointKurt,Classification
0,movie1candledeno_Tracks_DM_1,10.900693,0.828096,0.780058,0.681895,0.047414,0.057740,1.707145,DM
1,movie1candledeno_Tracks_DM_2,13.095078,0.888593,0.966746,0.685196,0.019226,0.165591,1.767373,DM
2,movie1candledeno_Tracks_DM_3,0.953884,1.000000,NaN,0.693147,NaN,0.000000,1.000000,DM
3,movie1candledeno_Tracks_DM_4,8.906587,0.595560,0.682262,0.504435,0.010121,0.297802,1.712182,DM
4,movie1candledeno_Tracks_DM_5,30.413202,0.774028,0.858904,0.676339,0.011720,0.046033,1.818068,DM


In [71]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(features_tracks, test_size=0.2, random_state=42)

In [72]:
train_set.head ()

,Id,netDisp,efficiency,straightness,Asym,bending,pointSkew,pointKurt,Classification
963,movienonoise2_Tracks_di_171,0.000000,0.000000,-0.500000,0.693147,0.100000,0.183519,3.935510,D
762,movienonoise1_Tracks_Di_167,1.000000,0.022222,-0.101015,0.693147,0.101015,2.267787,6.142857,D
334,movie2candledeno_Tracks_DM_125,6.432394,0.679336,0.592819,0.640875,0.045316,0.146892,2.063225,DM
889,movienonoise2_Tracks_di_97,0.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,D
529,movie3candledeno_Tracks_DM_128,0.861252,1.000000,NaN,0.693147,NaN,0.000000,1.000000,DM


In [73]:
def data_categorization(df, col):
    #df['Cat_classification'] = df[col]
    labels = df[col].astype('category').cat.categories.tolist()
    replace_map_comp = {col: {k: v for k, v in zip(
        labels, list(range(1, len(labels) + 1)))}}
    df.replace(replace_map_comp, inplace=True)
    return df

def cleaning_columns (df,column_target):
    df = df.drop(column_target, axis=1)
    return df 


In [74]:
training_cleaned_data = (train_set.pipe(data_categorization, col='Classification').pipe (cleaning_columns,'Id')
                    )
training_cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 788 entries, 963 to 102
Data columns (total 8 columns):
netDisp           788 non-null float64
efficiency        765 non-null float64
straightness      709 non-null float64
Asym              765 non-null float64
bending           709 non-null float64
pointSkew         765 non-null float64
pointKurt         765 non-null float64
Classification    788 non-null int64
dtypes: float64(7), int64(1)
memory usage: 55.4 KB


In [75]:
def getting_working_vars (df,column_target):
    X_train = df.drop(column_target, axis=1)
    # You can covert the target variable to numpy
    y_train = df[column_target].values
   
    return X_train,y_train

X,Y = getting_working_vars (training_cleaned_data,['Classification'])

print(len(X))
print(list(X))

788
['netDisp', 'efficiency', 'straightness', 'Asym', 'bending', 'pointSkew', 'pointKurt']


In [76]:
from sklearn.base import BaseEstimator, TransformerMixin
class FeatureSelector(BaseEstimator, TransformerMixin):
    # Class Constructor
    def __init__(self, feature_names):
        self.feature_names = feature_names

        # Return self nothing else to do here

    def fit(self, X, y=None):
        return self

        # Method that describes what we need this transformer to do

    def transform(self, X, y=None):
        return X[
            self.feature_names].values  # np.asarray(X[self.feature_names]).astype(str)#np.c_[X]#X[self.feature_names]#pd.DataFrame (X,columns= self.feature_names)


In [77]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

num_feature_name =list(X)
cat_feature_name = ["Classification"]
num_feature_name
full_pipeline = Pipeline([
        ('selector', FeatureSelector(num_feature_name)),
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', MinMaxScaler()),
    ])

In [63]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
#classifier = LogisticRegression;


# The full pipeline as a step in another pipeline with an estimator as the final step
full_pipeline_m = Pipeline(steps=[('full_pipeline', full_pipeline),

                                ('model', LogisticRegression(class_weight='balanced'))])


param_grid = [{'model': [LogisticRegression()],
              'model__penalty': ['l1', 'l2'],
              'model__C': [10, 25, 30, 40],
              'model__class_weight':['balanced'],
              'full_pipeline__imputer__strategy':['mean','median','most_frequent'],
              },
              {"model": [RandomForestClassifier()],
               "model__n_estimators": [10, 100, 1000],
               "model__max_depth":[25,30,None],
               "model__min_samples_leaf":[10,15,100],
               "model__max_leaf_nodes": [2, 5,10],
               'full_pipeline__imputer__strategy':['mean','median','most_frequent'],
              }]

scoring = ['precision_macro', 'recall_macro', 'f1_macro',
               'balanced_accuracy']

grid_search = GridSearchCV(full_pipeline_m, param_grid, cv=5,
                                     n_jobs=-1, verbose=1, scoring=scoring, refit='recall_macro')
    #
grid_search.fit(X,Y)

Fitting 5 folds for each of 267 candidates, totalling 1335 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 301 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 551 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 901 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 1335 out of 1335 | elapsed:  7.8min finished
/Users/ruddirodriguez/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('full_pipeline',
                                        Pipeline(memory=None,
                                                 steps=[('selector',
                                                         FeatureSelector(feature_names=['netDisp',
                                                                                        'efficiency',
                                                                                        'straightness',
                                                                                        'Asym',
                                                                                        'bending',
                                                                                        'pointSkew',
                                                                                        'pointKurt'])),
          

In [78]:
test_cleaned_data = (test_set.pipe(data_categorization, col='Classification').pipe (cleaning_columns,'Id')
                    )
test_cleaned_data.info()
X_test,Y_test = getting_working_vars (test_cleaned_data,['Classification'])

print(len(X_test))
print(list(X_test))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197 entries, 613 to 717
Data columns (total 8 columns):
netDisp           197 non-null float64
efficiency        193 non-null float64
straightness      172 non-null float64
Asym              193 non-null float64
bending           172 non-null float64
pointSkew         193 non-null float64
pointKurt         193 non-null float64
Classification    197 non-null int64
dtypes: float64(7), int64(1)
memory usage: 13.9 KB
197
['netDisp', 'efficiency', 'straightness', 'Asym', 'bending', 'pointSkew', 'pointKurt']


In [82]:
from sklearn import metrics
print(grid_search.best_params_)

predicted = grid_search.predict(X_test)
#predicted
#len(y_train)
print(metrics.accuracy_score(Y_test, predicted))
print(metrics.classification_report(Y_test, predicted))

{'full_pipeline__imputer__strategy': 'most_frequent', 'model': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=25, max_features='auto', max_leaf_nodes=10,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False), 'model__max_depth': 25, 'model__max_leaf_nodes': 10, 'model__min_samples_leaf': 10, 'model__n_estimators': 100}
0.9695431472081218
              precision    recall  f1-score   support

           1       0.94      0.97      0.96        70
           2       0.98      0.97      0.98       127

    accuracy                           0.97       197
   macro avg       0.96      0.97      0.97       197
weighted avg       0.97      0.97      0.97  